## Import functions and define universal values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0,'./code')
import run_higher_sph_harm
import plot_utils

lcName='W18_NIRISS_spec_priors_bin25_test'
planetparams={}
planetparams['t0']=2459802.4078798565 #units of days
planetparams['per']=0.941452382 #units of days
planetparams['a_abs']=0.0218 #units of AU
planetparams['inc']=84.35320 #units of degrees
planetparams['ecc']=0.0 #unitless
planetparams['w']=269. #units of degrees
planetparams['rprs']=0.09783 #unitless ratio
planetparams['ars']=3.48023 #unitless ratio
planetparams['t_sec']=2459802.8786060475 #units of days

## Load in data and set up data dictionary

In [ ]:
homedir='/Users/megan/Documents/Projects/JWST_ERS/EclipseMap/'
specfile=np.load(homedir+'/real_spec/bin25/spec_lambin_25.npz')
time=specfile['arr_0']
waves=specfile['arr_1']
dlam=specfile['arr_2']#np.diff(waves)[0]*np.ones_like(waves)
fluxes=specfile['arr_3']+10**6.
errs=specfile['arr_4']
extent=np.zeros(2)
extent[0]=(np.min(time)-planetparams['t_sec'])/planetparams['per']*2.*np.pi-np.pi/2. #minimum observed point, in radians
extent[1]=(np.max(time)-planetparams['t_sec'])/planetparams['per']*2.*np.pi+np.pi/2. #maximum observed point, in radians
print(extent*180./np.pi)

plt.figure()
plt.errorbar(time,fluxes[3,:],yerr=errs[3,:],color='k',marker='.',linestyle='none')
plt.show()

datadict={"time (days)":time, "wavelength (um)":waves, "flux (ppm)": fluxes, "flux err (ppm)": errs}


## Perform eigencurve fitting at each wavelength

In [ ]:
#Test fitting with spherical harmonics to order=2,3,4
#Afew>=20 will fit for the correct number of coefficients
saveDir = homedir+"/data/sph_harmonic_coefficients_full_samples/" + lcName
firstfits = homedir+"/data/besteigenlists/W18_NIRISS_spec_lambin_25_prefit_all/"
run_higher_sph_harm.run_lc_fit(datadict,planetparams,norder=[5,3,6,5,5,5,3,6,5,4,4,4,5,4,5,3,4,4,3,3,4,5,5,6,4]\
                                    ,lcName=lcName,\
                                    saveDir=saveDir,firstfits=firstfits,afew=[3,3,5,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2],\
                                    burnin=700,nsteps=7000,plot=False,strict=False,nonegs=True)
    
#[5,3,6,5,5,5,3,6,5,4,4,4,5,4,5,3,4,4,3,3,4,5,5,6,4] for bin 25 fiducial norder
#[3,3,5,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2] for bin 25 fiducial afew
#[4,6,5,3,4] for bin 05 fiducial norder
#[3,3,6,2,3] for bin 05 fiducial afew
#[5,5,3,3,5,6,6,3] for optbin 08 fiducial norder
#[2,4,4,2,2,2,2,2] for optbin 08 fiducial afew
#[5,5,5,3,4,6,6,3] for optbin 08 alternate norder
#[2,4,3,2,3,2,2,2] for optbin 08 alternate afew
#[5,3,4,4,5,5,3,6,5,4,4,4,5,3,4,4,4,4,3,3,4,5,5,6,4] for bin 25 alternate norder
#[3,2,4,4,2,2,2,2,2,2,2,2,2,3,3,3,2,2,3,2,2,2,2,2,2] for bin 25 alternate afew


## Plot 2D maps: Convert from spherical harmonics to maps

In [ ]:
f=np.load('./data/sph_harmonic_coefficients_full_samples/W18_NIRISS_spec_priors_bin25_newcode/spherearray_deg_list.npz',allow_pickle=True)
outDictionary=f['arr_0'].tolist()
eigensamples=outDictionary['eigencurve coefficients']
vals=np.zeros(len(eigensamples))
for i in np.arange(len(eigensamples)):
    vals[i]=np.shape(eigensamples[i])[0]
numsamp=np.min(vals)
print(vals)

In [ ]:
dataDir=saveDir+'/'
dataDir="./data/sph_harmonic_coefficients_full_samples/W18_NIRISS_spec_priors_bin25_newcode/"
deglist=[5,5,3,3,5,6,6,3]
eigenlist=[2,4,4,2,2,2,2,2]
outputDir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
londim=360 #360 is one point per degree 120 40
latdim=180 #180 is one point per degree 60 20
outputpath = plot_utils.retrieve_map_full_samples\
    (planetparams,dataDir,outputDir,londim,latdim,afew=eigenlist,degree=deglist,nrandom='all',isspider=False)


In [ ]:
#calculate minimum and maximum flux
outputpath='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
minflux,maxflux=plot_utils.calc_minmax(outputpath,extent)
print(minflux,maxflux)
#minflux=-1.7370372118450387e-05 for bin25 fiducial
#maxflux=0.0013331443311678265 for bin25 fiducial
#minflux = -1.482361330749393e-05 for bin5 fiducial
#maxflux = 0.0012410430842583183 for bin5 fiducial
#minflux = -1.5865805733546162e-05 for optbin8 fiducial
#maxflux = 0.0011568188607426996 for optbin8 fiducial
#minflux = -1.4375074707538006e-05 for bin25 alt
#maxflux = 0.0013035095538206328 for bin25 alt
#minflux = -1.5864596749491696e-05 for bin8 alt
#maxflux = 0.0011565753038259305 for bin8 alt
#minflux = -1.4179303302826664e-05 for bin25 newcode
#maxflux = 0.0013043111883153209 for bin 25 newcode

## Plot 2D Maps: Perform Plotting

In [ ]:
outputpath='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
minflux=0
maxflux=0.0013043111883153209
mapLowMedHigh = plot_utils.plot_retrieved_map(outputpath,waves,extent,minflux,maxflux,waveInd='Full',\
                                                          saveName='W18_spec_bin25')

In [ ]:
Tstar=6400.
rprs=planetparams['rprs']
savepath='./data/normalized_maps/deg_list_bin25_newcode/temp_maps/'
datadir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
#if reading in a model, the wavelengths must be in units of m and the flux must be in units of W/m^3/sr
smodel=np.loadtxt('./PHOENIX_W18_Teff_6435K_logg_4.35_corrunit.txt')
test=plot_utils.plot_map_in_temp(datadir,savepath,waves,dlam,rprs,extent,waveInd='Full',\
                                         saveName='W18_spec_bin25',startype='Temp',Tstar=Tstar,smodel=smodel)


## Calculate brightest point in longitude and latitude

In [ ]:
outputpath='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
min_lon=-20
max_lon=20
step_size=1
hotlats,hotlons = plot_utils.find_hotspot(outputpath,waves,min_lon,max_lon,step_size,waveInd='Full',\
                                                  saveName='W18_spec_bin25')


In [ ]:
from matplotlib import rc
rc('axes',linewidth=2)
plt.figure()
# plt.errorbar(waves,hotlats[:,1],yerr=[(hotlats[:,1]-hotlats[:,0]),(hotlats[:,2]-hotlats[:,1])],color='b',\
             #label='Latitude',marker='.',linestyle='none',markersize=15,linewidth=3)
plt.errorbar(waves,hotlons[:,2],yerr=[(hotlons[:,4]-hotlons[:,2]),(hotlons[:,2]-hotlons[:,0])],color='r',\
             label='Longitude',marker='.',linestyle='none',markersize=15,linewidth=3)
plt.tick_params(labelsize=20,axis="both",top=True,right=True,width=2,length=8,direction='in')
plt.legend(fontsize=15)
plt.xlabel('Wavelength [$\mu$m]',fontsize=20)
plt.ylabel('Hotspot Offset [$\degree$]',fontsize=20)
plt.tight_layout()
plt.savefig(outputpath+'hotspot_offset_longonly.pdf')
plt.show()
newfile=open('hotspot_offset_bin25.txt','w')
for i in np.arange(len(waves)):
    print(waves[i],hotlons[i,0],hotlons[i,1],hotlons[i,2],hotlons[i,3],hotlons[i,4],file=newfile)
newfile.close()

## Perform grouping

In [ ]:
dataDir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
ngroups=2 #FINDME: change ngroups to cluster the data into a different number of eigengroups
#If the kernel dies, use a smaller number of samples.
numsamp=500
eigenspec,eigenerr,grouping,fullplanetspec,fullplaneterr = plot_utils.find_groups(dataDir,waves,\
                            extent,numsamp,ngroups=ngroups,letters=True)
#FINDME: can change trySamples to set the number of MCMC draws you'd like to use to find the groups (larger number will make it run slower)

In [ ]:
#convert eigenspectra to temperatures
dataDir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
ngroups=4 #FINDME: change ngroups to cluster the data into a different number of eigengroups
#If the kernel dies, use a smaller number of samples.
numsamp=500
rprs=planetparams['rprs']
smodel=np.loadtxt('./PHOENIX_W18_Teff_6435K_logg_4.35_corrunit.txt')
Tstar=6400.
tempspec,temperr=plot_utils.group_spectra_in_temp(dataDir,numsamp,ngroups,dlam,rprs,startype='Temp',Tstar=Tstar,smodel=smodel)



In [ ]:
print(tempspec)
print(waves)
print(np.shape(tempspec))
print(np.shape(temperr))
# print(fullplanetspec,fullplaneterr)
newfile=open('temp_spec_25bins_4groups.txt','w')
for i in np.arange(np.shape(waves)[0]):
    print(waves[i],tempspec[0,i],temperr[0,i,0],temperr[0,i,1],tempspec[1,i],temperr[1,i,0],temperr[1,i,1],\
          tempspec[2,i],temperr[2,i,0],temperr[2,i,1],tempspec[3,i],temperr[3,i,0],temperr[3,i,1],file=newfile)
newfile.close()

In [ ]:
savedir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
ngroups=3
numsamp=500
plot_utils.do_hue_maps(savedir,numsamp,ngroups,hueType='group')



In [ ]:
savedir='./data/normalized_maps/deg_list_bin25_newcode/flux_maps/'
ngroups=3
numsamp=500
plot_utils.do_hue_maps(savedir,numsamp,ngroups,hueType='flux')

